<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🔨👨‍⚖️ תהליך הסבת שכבת גבולות פסקי דין לבנק"ל מודרני 🔨👨‍⚖️ 

In [ ]:
print('8. Judgments borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from configs import CNFG
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  8.a Reading inputs')

<h1 style="text-align:right;font-size:100%">
שכבת גבולות פס"ד קיימת

In [ ]:
GVUL_PSAK_DIN_cols = ['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR', 'geometry']

GVUL_PSAK_DIN = gpd.read_file(CNFG.CADSDE + 'GVUL_PSAK_DIN_NEW1.gdb', layer = 'GVUL_PSAK_DIN_NEW1', include_fields = GVUL_PSAK_DIN_cols)

del GVUL_PSAK_DIN_cols

<h1 style="text-align:right;font-size:100%">
טבלת פס"ד

In [ ]:
orcale_cols = ['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR', 'NAME', 'SOURCE', 'GUSH_NUM', 'GUSH_SUFFIX', 'STATUS', 'STATUS_DATE', 'START_DATE']

oracle_df = pd.read_csv(CNFG.Oracle + 'psak_din.csv', encoding = 'cp1255', usecols = orcale_cols)

del orcale_cols

<h1 style="text-align:right;font-size:100%">
שכבת גושים מודרנית

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'BlockUniqueID'])

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  8.b Cleaning data')

In [ ]:
print('  8.c Data conversion')

<h1 style="text-align:right;font-size:125%">
:עיבוד שכבת גבולות פס"ד
<h1 style="text-align:right;font-size:100%">
הוספת שדה של שם התכנית לפי מספר ושנת פס"ד

In [ ]:
GVUL_PSAK_DIN[['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR']] = GVUL_PSAK_DIN[['PSAK_DIN_ID', 'PSAK_DIN_NUM', 'PSAK_DIN_YEAR']].astype(int)

GVUL_PSAK_DIN['ProcessName'] = GVUL_PSAK_DIN['PSAK_DIN_NUM'].astype(str) + '-' + GVUL_PSAK_DIN['PSAK_DIN_YEAR'].astype(str)
GVUL_PSAK_DIN.drop(columns = ['PSAK_DIN_NUM', 'PSAK_DIN_YEAR'], inplace=True)

<h1 style="text-align:right;font-size:125%">
:עיבוד טבלת פס"ד
<h1 style="text-align:right;font-size:100%">
הוספת שדה של שם התכנית לפי מספר ושנת פס"ד
<h1 style="text-align:right;font-size:100%">
הגדרת שדות של תאריך כשדות מסוג תאריך

In [ ]:
oracle_df['ProcessName'] = oracle_df['PSAK_DIN_NUM'].astype('str') + '-' + oracle_df['PSAK_DIN_YEAR'].astype('str')

In [ ]:
oracle_df['STATUS_DATE'] = pd.to_datetime(oracle_df['STATUS_DATE'], format = '%d/%m/%Y %H:%M:%S')

oracle_df['START_DATE'] = pd.to_datetime(oracle_df['START_DATE'], format = '%d/%m/%Y %H:%M:%S')

<h1 style="text-align:right;font-size:125%">
:עיבוד שכבת גושים מודרנית
<h1 style="text-align:right;font-size:100%">
יצירת שדה מקשר לטובת איחוד עם שכבת גושים והשגת מזהה גוש בהמשך   

In [ ]:
oracle_df['GUSH_SUFFIX'] = oracle_df['GUSH_SUFFIX'].fillna(0)
oracle_df[['GUSH_NUM', 'GUSH_SUFFIX']] = oracle_df[['GUSH_NUM', 'GUSH_SUFFIX']].astype(int)
oracle_df['BlockName'] = oracle_df['GUSH_NUM'].astype(str) + '/' + oracle_df['GUSH_SUFFIX'].astype(str)

<h1 style="text-align:right;font-size:125%">
מיזוג טבלאות 

In [ ]:
merge = GVUL_PSAK_DIN.merge(oracle_df, on = ['PSAK_DIN_ID', 'ProcessName'], how = 'outer').sort_values('ProcessName')

In [ ]:
merge = merge.merge(Blocks, left_on = 'BlockName', right_on = 'Name', how = 'left')
merge = merge.drop_duplicates(keep = 'first')

del [GVUL_PSAK_DIN, oracle_df, Blocks]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('  8.d Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("CPBUniqueID", int),
                        ("BlockUniqueID", int),
                        ("ProcessType", int),
                        ("MapYear", int),
                        ("ProcessName", str),
                        ("GeodeticNetwork", int),   #לא קיים שדה לייבוא
                        ("Status", int),
                        ("PlanName", str),
                        #("SurveyorLicenseID", int), #אין מודד בתהליכי פס"ד
                        ("DataSource", int),
                        ("geometry", int)
                        ])


Judgments = gpd.GeoDataFrame(np.empty(0, dtype = new_columns),
                             crs = 2039,
                             geometry = 'geometry')

del new_columns

<h1 style="text-align:right;font-size:100%">
    איכלוס שדות

In [ ]:
Judgments['BlockUniqueID'] = merge['BlockUniqueID'].astype('Int64')


Judgments['ProcessType'] = 3 # פסק דין


Judgments['MapYear'] = merge['START_DATE'].dt.year
Judgments['MapYear'] = Judgments['MapYear'].astype('Int64')


Judgments['ProcessName'] = merge['ProcessName'].astype(str)


statusDict = {1 : 11,    #בטיפול האגף - בטיפול האגף
              2 : 12,    #בבדיקת הממונה על המרשם - בבדיקת הממונה על המרשם
              3 : 13,    # מאושר - מאושר
              4 : 36,    # מבוטל - מבוטל
              None : 36} #ללא ערך - מבוטל
Judgments['Status'] = merge['STATUS'].map(statusDict)
Judgments['Status'] = Judgments['Status'].fillna(36)
Judgments['Status'] = Judgments['Status'].astype(int)


Judgments['PlanName'] = merge['NAME']


Judgments['GeodeticNetwork'] = 0


#Judgments['SurveyorLicenseID'] = None


Judgments['DataSource'] = 5 #CAD


Judgments['geometry'] = merge['geometry']


Judgments['CPBUniqueID'] = merge['PSAK_DIN_ID'] + 3000000
Judgments.dropna(subset = ['CPBUniqueID'], inplace = True)
Judgments['CPBUniqueID'] = Judgments['CPBUniqueID'].astype(int)

In [ ]:
Judgments.sort_values('CPBUniqueID', inplace=True)

del merge

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
Judgments_geom = Judgments[~Judgments['geometry'].isna()]
Judgments_no_geom = Judgments[Judgments['geometry'].isna()]

In [ ]:
print('  8.e Exporting results to product folder')

In [ ]:
no_geom_csv_dir = CNFG.CSVPath + 'Judgments_no_geom.csv'
json_dir        = CNFG.JsonPath + 'Judgments_geom.geojson'


Judgments_no_geom.drop(columns = ['geometry']).to_csv(no_geom_csv_dir, encoding = 'cp1255', index = False)
Judgments_geom.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([CNFG.BatchPath + 'Judgments-Save as Feature Class.bat']);